# DeepLabCut - using for Hard Pose Dataset

This notebook illustrates how to:
1. Get started in the terminal
2. Create / Load / Configure the project
3. Extract frames from videos
4. Label the extracted frames
5. Create training dataset
6. Train the network
7. Evaluate the network
8. Analyze the video
9. Plot the trajectories
10. Create an automatically labeled video

Note: DeepLabCut should install before running these srcipts. Refer [How to Install DeepLabCut](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/installation.md)

## 1. Get started in the terminal

- To begin, (windows) navigate to anaconda prompt and right-click to "open as admin ", or (unix/MacOS) simply launch "terminal" on your computer
- Next, in Terminal, go to the folder where you run your project by command: ``cd [C:\Users\YourUserName\Desktop\DeepLabCut\conda-environments]``
- Then, run ``activate DEEPLABCUT``
- Finally, open Jupiter lab by command `` jupyter-lab``
In Jupyter Lab, following the next steps

Note: We assume you have DeepLabCut installed.

In [4]:
# Importing the toolbox
import deeplabcut

## 2. Create / Load / Configure the project

### Create a project
If this is the first time you run project, using create_new_project function to create a new project directory.
Project is identified by:
- Name of the project
- Name of the experimenter
- Date at creation

Video is the full path of the videos that are (initially) used to create the training dataset.

Other parameters: copy_videos=True/False, multianimal=True/False.

The function returns the path of the config file that was just created from the below steps.

In [5]:
task='HardPose' # Enter the name of your experiment Task
experimenter='DoanVo' # Enter the name of the experimenter
video=['videos/HardPose.mp4'] # Enter the paths of your videos OR FOLDER you want to grab frames from.

path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=True) 

Project "C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\Test-DoanVo-2021-12-22" already exists!


### Load a project
If project created, you just load project to get config_path and videofile_path variables

Note: check config.yaml file to make sure the working_directory is corect.

In [5]:
#If you're loading an already created project, just set config_path and videofile_path variable
import os
from pathlib import Path
path_config_file = os.path.join(os.getcwd(),'HardPose-DoanVo-2021-12-21\config.yaml')
videofile_path = os.path.join(os.getcwd(),'HardPose-DoanVo-2021-12-21\\videos\\HardPose.mp4') 
print(videofile_path)
print(path_config_file)

C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\HardPose-DoanVo-2021-12-21\videos\HardPose.mp4
C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\HardPose-DoanVo-2021-12-21\config.yaml


### Configure a project
Open the `config.yaml` file. You can edit various parameters, in particular you must add the list of bodyparts (or points of interest) that you want to track. This is a crucial step.
In this project, there are 17 joints on body parts tracked as below

```python
bodyparts:
- left_ankle
- left_knee
- left_femoral
- right_femoral
- right_knee
- right_ankle
- left_wrist
- left_elbow
- left_shoulder
- left_chest
- right_chest
- right_shoulder
- right_elbow
- right_wrist
- hip
- abdominal
- head
start: 0
stop: 1
numframes2pick: 20

#Plotting configuration
skeleton: []
skeleton_color: black
pcutoff: 0.01
dotsize: 12
alphavalue: 0.7
colormap: rainbow
```

### 3. Extract frames
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

The `numframes2pick: [n]` parameter in `config.yaml` file will automatically select n frames from the video you provide. 

However, you can also do this step manually so that you can select a variety of frames with a variety of behaviors you want to analyze.

In [ ]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file) 

### 4. Label the extracted frames
This step is intended to label the joints of the body part (which are already set in `config.yaml`).

You can choose to do this step manually, with the [GUI] (https://github.com/DeepLabCut/DeepLabCut/blob/master/deeplabcut/gui/labeling_toolbox.py) provided by DeepLabCut, or any other tool that supports labeling.

After this step, the labeled images will be created and placed in the **labeled-data** folder of the project.

You can also copy labeled images of the same video to be analyzed into this folder.

In this project you can find the sample labeled image set for the Inter Pose dataset in the **labeled-data** folder

Note: Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function ‘check_labels’ to do so.


In [ ]:
deeplabcut.check_labels(config_path, visualizeindividuals=True)

### 5.  Create training dataset
There are several networks you can pick, the default is `resnet-50`, you also use resnet-101 for this step.

This function generates the training data information for network training based on the pandas dataframes that hold label information.

After running this script the training dataset is created and saved in the project directory under the subdirectory **training-datasets**

This function also creates new subdirectories under **dlc-models** and appends the project `config.yaml` file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. 

In [29]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_101', augmenter_type='imgaug')

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([19, 11, 17, 15,  7,  3,  2,  6, 12, 14,  9, 13,  5,  4,  1, 16,  0,
          10,  8]),
   array([18])))]

### 6. Train a network
This function trains the network for a specific shuffle of the training dataset.

It is recommended to train the networks for thousands of iterations until the loss plateaus (typically around 500,000) if you use batch size 1.

The variables `display_iters` and `save_iter`s in the `pose_cfg.yaml` file allows the user to alter how often the loss is displayed and how often the weights are stored.

This function will run until you stop it (CTRL+C) or when it hits `maxiters` parameter

In [30]:
deeplabcut.train_network(path_config_file, allow_growth=True, shuffle=1, displayiters=10,saveiters=50,maxiters=5000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16]],
 'all_joints_names': ['left_ankle',
                      'left_knee',
                      'left_femoral',
                      'right_femoral',
                      'right_knee',
                      'right_ankle',
                      'left_wrist',
                      'left_elbow',
                      'left_shoulder',
                      'left_chest',
                      'right_chest',
                      'right_shoulder',
                      'right_elbow',
                      'right_wrist',
                      'hip',
                      'abdominal',
                      'head'],
 'alpha_r': 0

Selecting single-animal trainer
Batch Size is 1


C:\Users\kdo6\Anaconda3\envs\DEEPLABCUT\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_101
Max_iters overwritten as 5000
Display_iters overwritten as 10
Save_iters overwritten as 50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\kdo6\\Downloads\\HumanMotionCapture\\DeepLab_Human\\conda-environments\\Test-DoanVo-2021-12-22\\dlc-models\\iteration-0\\TestDec22-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_pr

iteration: 10 loss: 0.5617 lr: 0.005
iteration: 20 loss: 0.0578 lr: 0.005
iteration: 30 loss: 0.0328 lr: 0.005
iteration: 40 loss: 0.0277 lr: 0.005
iteration: 50 loss: 0.0286 lr: 0.005
iteration: 60 loss: 0.0275 lr: 0.005
iteration: 70 loss: 0.0246 lr: 0.005
iteration: 80 loss: 0.0258 lr: 0.005
iteration: 90 loss: 0.0229 lr: 0.005
iteration: 100 loss: 0.0203 lr: 0.005
iteration: 110 loss: 0.0194 lr: 0.005
iteration: 120 loss: 0.0225 lr: 0.005
iteration: 130 loss: 0.0192 lr: 0.005
iteration: 140 loss: 0.0188 lr: 0.005
iteration: 150 loss: 0.0232 lr: 0.005
iteration: 160 loss: 0.0229 lr: 0.005
iteration: 170 loss: 0.0250 lr: 0.005
iteration: 180 loss: 0.0210 lr: 0.005
iteration: 190 loss: 0.0233 lr: 0.005
iteration: 200 loss: 0.0212 lr: 0.005
iteration: 210 loss: 0.0191 lr: 0.005
iteration: 220 loss: 0.0200 lr: 0.005
iteration: 230 loss: 0.0235 lr: 0.005
iteration: 240 loss: 0.0237 lr: 0.005
iteration: 250 loss: 0.0216 lr: 0.005
iteration: 260 loss: 0.0211 lr: 0.005
iteration: 270 loss: 

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


### 7. Evaluate the trained network
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .csv file in a subdirectory under **evaluation-results**.

Setting plotting to True plots all the testing and training frames with the manual and predicted labels; these will be colored by body part type by default. 

In [32]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16]],
 'all_joints_names': ['left_ankle',
                      'left_knee',
                      'left_femoral',
                      'right_femoral',
                      'right_knee',
                      'right_ankle',
                      'left_wrist',
                      'left_elbow',
                      'left_shoulder',
                      'left_chest',
                      'right_chest',
                      'right_shoulder',
                      'right_elbow',
                      'right_wrist',
                      'hip',
                      'abdominal',
                      'head'],
 'batch_size'

Running  DLC_resnet101_TestDec22shuffle1_5000  with # of training iterations: 5000
Running evaluation ...


20it [01:44,  5.25s/it]


Analysis is done and the results are stored (see evaluation-results) for snapshot:  snapshot-5000
Results for 5000  training iterations: 95 1 train error: 41.61 pixels. Test error: 42.31  pixels.
With pcutoff of 0.01  train error: 41.61 pixels. Test error: 42.31 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


<IPython.core.display.Javascript object>

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:18<00:00,  1.10it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
Please check the results, then choose the best model (snapshot) for prediction. You can update the config.yaml file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise, consider adding more labeled-data and retraining the network (see DeepLabCut workflow Fig 2, Nath 2019)


### 8. Analyze the video
This function analyzes the new video. You can choose the best model from the evaluation results and specify the correct snapshot index for the variable `snapshotindex` in the `config.yaml` file. Otherwise, by default the most recent snapshot is used to analyse the video. 
The results are stored in hd5 file in the same directory where the video resides. If the flag `save_as_csv=True`, the data can also be exported in comma-separated values format (.csv), which in turn can be imported in many programs.

In [33]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.mp4', save_as_csv=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16]],
 'all_joints_names': ['left_ankle',
                      'left_knee',
                      'left_femoral',
                      'right_femoral',
                      'right_knee',
                      'right_ankle',
                      'left_wrist',
                      'left_elbow',
                      'left_shoulder',
                      'left_chest',
                      'right_chest',
                      'right_shoulder',
                      'right_elbow',
                      'right_wrist',
                      'hip',
                      'abdominal',
                      'head'],
 'batch_size'

Using snapshot-5000 for model C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\Test-DoanVo-2021-12-22\dlc-models\iteration-0\TestDec22-trainset95shuffle1
Starting to analyze %  C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\Test-DoanVo-2021-12-22\videos\InterPose.mp4
Loading  C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\Test-DoanVo-2021-12-22\videos\InterPose.mp4
Duration of video [s]:  21.63 , recorded with  29.95 fps!
Overall # of frames:  648  found with (before cropping) frame dimensions:  720 1280
Starting to extract posture


650it [1:01:32,  5.68s/it]                                                                                             


Saving results in C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\Test-DoanVo-2021-12-22\videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet101_TestDec22shuffle1_5000'

### 9. Plot the trajectories
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.
It creates a folder called **plot-poses** (in the directory of the video). The plots display the coordinates of body parts vs. time, likelihoods vs time, the x- vs. y- coordinate of the body parts, as well as histograms of consecutive coordinate differences. 

In [35]:
#%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file, videofile_path)

Loading  C:\Users\kdo6\Downloads\HumanMotionCapture\DeepLab_Human\conda-environments\Test-DoanVo-2021-12-22\videos\InterPose.mp4 and data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plots created! Please check the directory "plot-poses" within the video directory


### 10. Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.
 - If you want to create high-quality videos, pass save_frames=True
 - If you want to use the filtered data for a video or directory of filtered videos, pass filtered=True
 - If you want to create a video with only he "dots" plotted, pass keypoints_only=True:

In [8]:
deeplabcut.create_labeled_video?
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype='mp4', draw_skeleton = True, save_frames=True, trailpoints=10)

Signature:
deeplabcut.create_labeled_video(
    config,
    videos,
    videotype='avi',
    shuffle=1,
    trainingsetindex=0,
    filtered=False,
    fastmode=True,
    save_frames=False,
    keypoints_only=False,
    Frames2plot=None,
    displayedbodyparts='all',
    displayedindividuals='all',
    codec='mp4v',
    outputframerate=None,
    destfolder=None,
    draw_skeleton=False,
    trailpoints=0,
    displaycropped=False,
    color_by='bodypart',
    modelprefix='',
    track_method='',
)
Docstring:
   Labels the bodyparts in a video. Make sure the video is already analyzed by the function 'analyze_video'

   Parameters
   ----------
   config : string
       Full path of the config.yaml file as a string.

   videos : list
       A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.

   videotype: string, optional
       Checks for the extension of the video in case the input to the v